## Note générale

* DBpedia chapters dans différentes langues
* Propriétés etc.

In [1]:
### Librairies à installer dans l'environnement conda
# qui sera choisi pour exécuter le carnet
from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
### Librairies système
from importlib import reload
import sqlite3 as sql
import datetime
import sys

In [4]:
### Importer un module de fonctions crées ad hoc
# Le module 'sparql_functions.py' n'est rien d'autre qu'un fichier 
# qui contient des fonctions qui seront réutilisées souvent
##  ATTENTION : le fichier 'sparql_functions.py' doit se trouver 
#  dans un dossier qui se situe dans le chemin ('path') de recherche
#  qui est vu par le présent carnet Jupyter afin que l'importation 
#  fonctionne correctement

# Add parent directory to the path
sys.path.insert(0, '..')

### If you want to add the parent-parent directory,
sys.path.insert(0, '../..')


import sparql_functions as spqf

## si on modifie fichier, il faut le recharger
# ce n'est généralement pas nécessaire donc commenté

# reload(spqf)

## Créer une table dans la base de données pour stocker les URI

Dans cette table on va mettre les URI se référant aux mêmes individus du monde, les mêmes _ressources_ selon l'expression technique, auxquels correspondent les URI de différents systèmes d'information.


Ici il s'agit des URI de Wikidata et de DBpedia, éventuellement dans les différentes versions linguistiques de DBPedia.


En général il vaut mieux toujours mettre l'URI d'un système en premier, par ex. Wikidata, et DBpedia ensuites.


On peut ainsi profiter de l'index _UNIQUE_ qui fait que si on reinsère une paire d'URI déjà existante, dans le même ordre, l'insertion sera ignorée et on évitera de produire des lignes en trop. 

In [8]:
### Requête SQL qui créer la table dans la BD

create_table_uri = """
CREATE TABLE same_as_uri (
    pk_same_as_uri INTEGER PRIMARY KEY AUTOINCREMENT,
    subject        TEXT,
    object         TEXT,
    notes          TEXT,
    UNIQUE (
        subject,
        object
    )
    ON CONFLICT IGNORE);
"""

In [5]:
### Requête qui écrase toute la table si on veut la refaire
# ATTENTION : cette mesure est irréversible
drop_table_uri = """
DROP TABLE same_as_uri;
"""

In [7]:
create_table_statement = """
CREATE TABLE IF NOT EXISTS statement (
    pk_statement INTEGER PRIMARY KEY AUTOINCREMENT,
    subject        TEXT,
    property      TEXT,
    object         TEXT,
    notes          TEXT,
    import_notes   TEXT,
    -- la prochaine ligne seulement si la table existe
    fk_sparql_query     INTEGER  REFERENCES sparql_query (pk_sparql_query) MATCH SIMPLE,
    UNIQUE (
        subject,
        property,
        object
    )
    ON CONFLICT IGNORE);
    """

In [6]:
### Se connecter à la base de données dans laquelle on va insérer
# le résultat de la requête SPARQL
cn = sql.connect('../../data/astronomers_import.db')
cn

In [10]:
### Exécuter une requête SQL
cur = cn.cursor()
try:
    l = cur.execute(create_table_uri)
except Exception as e:
    print('Error:', e)


Error: table same_as_uri already exists


In [108]:
### Cette instruction rend la modification persistante
# ATTENTION : la mesure est irrévocable, donc commenté par prudence
# Décommenter pour exécuter

# cn.commit()

In [11]:
### Fermer la connexion à la fin du travail
cn.close()

## Récupérer les données sameAs depuis le point d'accès Wikidata

In [12]:
## define SPARQL enpoint
endpoint = "https://query.wikidata.org/sparql"

In [14]:
q1 = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>

SELECT DISTINCT ?item ?uri_dbpedia
      WHERE {
      # SERVICE <https://query.wikidata.org/sparql>
      {
            
       {
        {?item wdt:P106 wd:Q169470}
        UNION
        {?item wdt:P106 wd:Q11063}
        UNION
        {?item wdt:P106 wd:Q155647}
        }
          
          ?item wdt:P31 wd:Q5;  # Any instance of a human.
              wdt:P569 ?birthDate.
        BIND(REPLACE(str(?birthDate), "(.*)([0-9]{4})(.*)", "$2") AS ?year)
        FILTER(xsd:integer(?year) > 1300 )
          # OPTIONAL {
      ?article schema:about ?item .
      ?article schema:inLanguage "en" .
     FILTER (SUBSTR(str(?article), 1, 25) = "https://en.wikipedia.org/")
      BIND (replace(str(?article), "https://en.wikipedia.org/wiki/", "http://dbpedia.org/resource/") AS ?uri_dbpedia)       

#  }
             }
             }
"""

In [15]:
### Executer la requête avec les fonctions de la librairie locale
qr = spqf.get_json_sparql_result(endpoint,q1)

In [16]:
### Stocker le résultat dans une variable liste et afficher
r = [l[:3] for l in spqf.sparql_result_to_list(qr)]
print(len(r))
r[:3]

13267


[['http://www.wikidata.org/entity/Q27323',
  'http://dbpedia.org/resource/Friedrich_Risner'],
 ['http://www.wikidata.org/entity/Q27374',
  'http://dbpedia.org/resource/Karl_Eduard_Zetzsche'],
 ['http://www.wikidata.org/entity/Q64839',
  'http://dbpedia.org/resource/Max_Wien']]

In [17]:
### Se connecter à la base de données dans laquelle on va insérer
# le résultat de la requête SPARQL
cn = sql.connect('../../data/astronomers_import.db')
# cn

In [20]:
### Insérer le nouvelles lignes
# ATTENTION : si elles existent déjà, dans le même ordre rien ne sera ajouté
# Noter que c'eset toute la liste qui est insérée
cur = cn.cursor()
cur.executemany('''INSERT INTO same_as_uri(subject,object) VALUES (?,?)''', r)
cn.commit()

## Récupérer les identifiants DBpedia d'une partie de la population et interroger DBpedia

In [21]:
### 
cur = cn.cursor()
l1 = cur.execute("""SELECT sau."object", sau.subject
FROM wdt_personne wp, same_as_uri sau 
WHERE sau.subject = wp.personUri
AND wp.birthYear > 1780""").fetchall()
# l1 = cur.execute("SELECT object FROM same_as_uri limit 200 offset 300").fetchall()

In [25]:
_l = [print(e[0]) for e in l1[:5]]

http://dbpedia.org/resource/David_Brewster
http://dbpedia.org/resource/Joseph_Johann_von_Littrow
http://dbpedia.org/resource/Giovanni_Plana
http://dbpedia.org/resource/Sim%C3%A9on_Denis_Poisson
http://dbpedia.org/resource/Henry_Atkinson_(scientist)


In [26]:
print(len(l1))

11992


In [27]:
## define SPARQL enpoint
endpoint_dbp = "https://dbpedia.org/sparql"

In [78]:
### Préparer la liste des individus qui servira de base à la requête
lq = []
i = 1

la = l1[1000:1300]
ll = len(la)

for e in la:
    if i < ll:
        lq.append('{<' + e[0] + '> ?p ?o} UNION ')
        i += 1
    else:
        lq.append('{ <' + e[0] + '> ?p ?o}')


print(len(lq))
lq[:3], lq[-1]

300


(['{<http://dbpedia.org/resource/Henry_Augustus_Rowland> ?p ?o} UNION ',
  '{<http://dbpedia.org/resource/Lu%C3%ADs_Cruls> ?p ?o} UNION ',
  '{<http://dbpedia.org/resource/Charles_S._Hastings> ?p ?o} UNION '],
 '{ <http://dbpedia.org/resource/Augusta_D%C3%A9jerine-Klumpke> ?p ?o}')

In [79]:
### Transformer la liste en chaîne de caractères
jlq = ''.join(lq)
# Ispecter les premières 500 caractères
print(jlq[:500])

{<http://dbpedia.org/resource/Henry_Augustus_Rowland> ?p ?o} UNION {<http://dbpedia.org/resource/Lu%C3%ADs_Cruls> ?p ?o} UNION {<http://dbpedia.org/resource/Charles_S._Hastings> ?p ?o} UNION {<http://dbpedia.org/resource/William_Frederick_Denning> ?p ?o} UNION {<http://dbpedia.org/resource/George_Mitchell_Seabroke> ?p ?o} UNION {<http://dbpedia.org/resource/Nils_Gustaf_Ekholm> ?p ?o} UNION {<http://dbpedia.org/resource/Temistocle_Zona> ?p ?o} UNION {<http://dbpedia.org/resource/Shelford_Bidwell>


In [80]:
### Préparer la requête SPARQl à exécuter
q2 = """
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbp: <http://dbpedia.org/property/>

SELECT ?p (COUNT(*) as ?eff)
WHERE {
 
 {
 
"""  + jlq + """

 }

}
GROUP BY ?p
ORDER BY DESC(?eff)
"""

In [81]:
### Executer la requête avec les fonctions de la librairie locale
qr = spqf.get_json_sparql_result(endpoint_dbp,q2)

In [83]:
r = [l for l in spqf.sparql_result_to_list(qr) if 'ontolo' in l[0]]
print(len(r))
import pprint
pprint.pprint(r[:50])

60
[['http://dbpedia.org/ontology/wikiPageWikiLink', '11968'],
 ['http://dbpedia.org/ontology/abstract', '2030'],
 ['http://dbpedia.org/ontology/wikiPageExternalLink', '888'],
 ['http://dbpedia.org/ontology/wikiPageLength', '260'],
 ['http://dbpedia.org/ontology/wikiPageID', '260'],
 ['http://dbpedia.org/ontology/wikiPageRevisionID', '260'],
 ['http://dbpedia.org/ontology/birthPlace', '217'],
 ['http://dbpedia.org/ontology/knownFor', '197'],
 ['http://dbpedia.org/ontology/thumbnail', '191'],
 ['http://dbpedia.org/ontology/deathPlace', '172'],
 ['http://dbpedia.org/ontology/birthDate', '135'],
 ['http://dbpedia.org/ontology/deathDate', '126'],
 ['http://dbpedia.org/ontology/academicDiscipline', '119'],
 ['http://dbpedia.org/ontology/almaMater', '106'],
 ['http://dbpedia.org/ontology/award', '105'],
 ['http://dbpedia.org/ontology/institution', '55'],
 ['http://dbpedia.org/ontology/doctoralStudent', '54'],
 ['http://dbpedia.org/ontology/occupation', '43'],
 ['http://dbpedia.org/ontology/n

### Traitement de l'ensemble

In [84]:
ia = 0
ib = 300

l_final = []

while ib < len(l1):
    lq = []
    i = 1

    la = l1[ia:ib]
    ll = len(la)

    for e in la:
        if i < ll:
            lq.append('{<' + e[0] + '> ?p ?o} UNION ')
            i += 1
        else:
            lq.append('{ <' + e[0] + '> ?p ?o}')

    jlq = ''.join(lq)
    q2 = """
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbp: <http://dbpedia.org/property/>

    SELECT ?p (COUNT(*) as ?eff)
    WHERE {
    
    {
    
    """  + jlq + """

    }

    }
    GROUP BY ?p
    ORDER BY DESC(?eff)
    """

    qr = spqf.get_json_sparql_result(endpoint_dbp,q2)
    r = [l for l in spqf.sparql_result_to_list(qr) if 'ontolo' in l[0]]
    l_final += r

    ia += 300
    ib += 300
    print(ib)

600
900
1200
1500
1800
2100
2400
2700
3000
3300
3600
3900
4200
4500
4800
5100
5400
5700
6000
6300
6600
6900
7200
7500
7800
8100
8400
8700
9000
9300
9600
9900
10200
10500
10800
11100
11400
11700
12000


In [85]:
print(len(l_final))

2189


In [93]:
df_lf = pd.DataFrame(l_final, columns=['p', 'eff'])
df_lf.head(), df_lf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2189 entries, 0 to 2188
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   p       2189 non-null   object
 1   eff     2189 non-null   object
dtypes: object(2)
memory usage: 34.3+ KB


(                                                  p    eff
 0      http://dbpedia.org/ontology/wikiPageWikiLink  12858
 1              http://dbpedia.org/ontology/abstract   2537
 2  http://dbpedia.org/ontology/wikiPageExternalLink   1055
 3            http://dbpedia.org/ontology/birthPlace    263
 4        http://dbpedia.org/ontology/wikiPageLength    259,
 None)

In [94]:
df_lf['eff'] = df_lf['eff'].astype('int64')

In [106]:
ldf = df_lf.groupby(by='p').sum().sort_values(by='eff', ascending=False)
ldf.head(30)


,eff
p,
http://dbpedia.org/ontology/wikiPageWikiLink,441882
http://dbpedia.org/ontology/abstract,56882
http://dbpedia.org/ontology/wikiPageExternalLink,31014
http://dbpedia.org/ontology/wikiPageID,10324
http://dbpedia.org/ontology/wikiPageLength,10324
http://dbpedia.org/ontology/wikiPageRevisionID,10324
http://dbpedia.org/ontology/birthPlace,8391
http://dbpedia.org/ontology/almaMater,6543
http://dbpedia.org/ontology/knownFor,6429


## Récupérer les universités des études

In [108]:
### Préparer la liste des individus tout en ajoutant l'URI wikidata
lq = []
i = 1

la = l1[1000:1300]
ll = len(la)

for e in la:
    if i < ll:
        lq.append('{BIND( <'+ e[1]+ '> as ?wd_item). <' + e[0] + '> dbo:almaMater ?am} UNION ')
        i += 1
    else:
        lq.append('{BIND( <'+ e[1]+ '> as ?wd_item). <' + e[0] + '> dbo:almaMater ?am}')


print(len(lq))
lq[:3], lq[-1]

300


(['{BIND( <http://www.wikidata.org/entity/Q505197> as ?wd_item). <http://dbpedia.org/resource/Henry_Augustus_Rowland> dbo:almaMater ?am} UNION ',
  '{BIND( <http://www.wikidata.org/entity/Q2155466> as ?wd_item). <http://dbpedia.org/resource/Lu%C3%ADs_Cruls> dbo:almaMater ?am} UNION ',
  '{BIND( <http://www.wikidata.org/entity/Q15429143> as ?wd_item). <http://dbpedia.org/resource/Charles_S._Hastings> dbo:almaMater ?am} UNION '],
 '{BIND( <http://www.wikidata.org/entity/Q2870950> as ?wd_item). <http://dbpedia.org/resource/Augusta_D%C3%A9jerine-Klumpke> dbo:almaMater ?am}')

In [109]:
jlq = ''.join(lq)
print(jlq[:500])

{BIND( <http://www.wikidata.org/entity/Q505197> as ?wd_item). <http://dbpedia.org/resource/Henry_Augustus_Rowland> dbo:almaMater ?am} UNION {BIND( <http://www.wikidata.org/entity/Q2155466> as ?wd_item). <http://dbpedia.org/resource/Lu%C3%ADs_Cruls> dbo:almaMater ?am} UNION {BIND( <http://www.wikidata.org/entity/Q15429143> as ?wd_item). <http://dbpedia.org/resource/Charles_S._Hastings> dbo:almaMater ?am} UNION {BIND( <http://www.wikidata.org/entity/Q594112> as ?wd_item). <http://dbpedia.org/resou


In [114]:
q2 = """
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?wd_item ?am
WHERE {
 
 {
 
"""  + jlq + """

 }

}
"""

In [115]:
### Executer la requête avec les fonctions de la librairie locale
qr = spqf.get_json_sparql_result(endpoint_dbp,q2)

In [116]:
r = [l[:3] for l in spqf.sparql_result_to_list(qr)]
print(len(r))
import pprint
pprint.pprint(r[:10])

106
[['http://www.wikidata.org/entity/Q505197',
  'http://dbpedia.org/resource/Rensselaer_Polytechnic_Institute'],
 ['http://www.wikidata.org/entity/Q15429143',
  'http://dbpedia.org/resource/Yale_University'],
 ['http://www.wikidata.org/entity/Q128823',
  'http://dbpedia.org/resource/University_College_London'],
 ['http://www.wikidata.org/entity/Q128823',
  'http://dbpedia.org/resource/Royal_College_of_Science'],
 ['http://www.wikidata.org/entity/Q64481',
  'http://dbpedia.org/resource/University_of_Leipzig'],
 ['http://www.wikidata.org/entity/Q84783',
  'http://dbpedia.org/resource/University_of_Vienna'],
 ['http://www.wikidata.org/entity/Q1627897',
  'http://dbpedia.org/resource/Trinity_College,_Cambridge'],
 ['http://www.wikidata.org/entity/Q446620',
  'http://dbpedia.org/resource/Yale_College'],
 ['http://www.wikidata.org/entity/Q5539344',
  'http://dbpedia.org/resource/University_of_St_Andrews'],
 ['http://www.wikidata.org/entity/Q5539344',
  "http://dbpedia.org/resource/Christ's

### Traitement de l'ensemble

In [145]:
ia = 0
ib = 300

l_final = []

while ib < len(l1):
    lq = []
    i = 1

    la = l1[ia:ib]
    ll = len(la)

    for e in la:
        if i < ll:
            lq.append('{BIND( <'+ e[1]+ '> as ?wd_item). <' + e[0] + '> dbo:almaMater ?am} UNION ')
            i += 1
        else:
            lq.append('{BIND( <'+ e[1]+ '> as ?wd_item). <' + e[0] + '> dbo:almaMater ?am}')

    jlq = ''.join(lq)
    q2 = """
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbp: <http://dbpedia.org/property/>
    PREFIX dbo: <http://dbpedia.org/ontology/>

    SELECT ?wd_item ?am ?am_label ?geo_point
    WHERE {
    
    {
    
    """  + jlq + """

    }

   {?am rdfs:label ?am_label.
    FILTER(lang(?am_label)="en")}
    OPTIONAL {?am <http://www.georss.org/georss/point> ?geo_point}

    }
    """

    qr = spqf.get_json_sparql_result(endpoint_dbp,q2)
    r = [l for l in spqf.sparql_result_to_list(qr)]
    l_final += r

    ia += 300
    ib += 300
    print(ib-300, ib)

300 600
600 900
900 1200
1200 1500
1500 1800
1800 2100
2100 2400
2400 2700
2700 3000
3000 3300
3300 3600
3600 3900
3900 4200
4200 4500
4500 4800
4800 5100
5100 5400
5400 5700
5700 6000
6000 6300
6300 6600
6600 6900
6900 7200
7200 7500
7500 7800
7800 8100
8100 8400
8400 8700
8700 9000
9000 9300
9300 9600
9600 9900
9900 10200
10200 10500
10500 10800
10800 11100
11100 11400
11400 11700
11700 12000


In [146]:
print(len(l_final))
l_final[:3]

6532


[['http://www.wikidata.org/entity/Q168468',
  'http://dbpedia.org/resource/University_of_Edinburgh',
  'University of Edinburgh',
  '55.94738888888889 -3.1871944444444447'],
 ['http://www.wikidata.org/entity/Q302590',
  'http://dbpedia.org/resource/Charles_University',
  'Charles University',
  '50.0884 14.4037'],
 ['http://www.wikidata.org/entity/Q373799',
  'http://dbpedia.org/resource/École_Polytechnique',
  'École Polytechnique',
  '']]

In [148]:
df_almaM = pd.DataFrame(l_final, columns=['wd_uri', 'almaM_uri', 'label', 'geocoord'])
df_almaM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6532 entries, 0 to 6531
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   wd_uri     6532 non-null   object
 1   almaM_uri  6532 non-null   object
 2   label      6532 non-null   object
 3   geocoord   6532 non-null   object
dtypes: object(4)
memory usage: 204.2+ KB


In [149]:
df_almaM.head(3)

,wd_uri,almaM_uri,label,geocoord
0,http://www.wikidata.org/entity/Q168468,http://dbpedia.org/resource/University_of_Edin...,University of Edinburgh,55.94738888888889 -3.1871944444444447
1,http://www.wikidata.org/entity/Q302590,http://dbpedia.org/resource/Charles_University,Charles University,50.0884 14.4037
2,http://www.wikidata.org/entity/Q373799,http://dbpedia.org/resource/École_Polytechnique,École Polytechnique,


In [ ]:
try:
    df_almaM.to_sql(con=cn, name='wdt_alma_mater_from_dbpedia', if_exists='fail')
except Exception as e:
    print('Error:', e)    

In [158]:
df_count= df_almaM.groupby(by='label').size().sort_values(ascending=False)
df_count.head(10)

label
Harvard University                       195
University of Cambridge                  181
Massachusetts Institute of Technology    164
Princeton University                     143
University of California, Berkeley       121
California Institute of Technology       118
University of Chicago                    113
Cornell University                       108
Columbia University                       97
Moscow State University                   81
dtype: int64